In [ ]:
import common_import 
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.transforms import NormalizeFeatures
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import json

In [ ]:
from torch_geometric.datasets import Planetoid

# Assuming your numpy array is named `data_array`
# data_array = np.loadtxt('data_mother/500label.csv', delimiter=',')
# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]
# data_array = np.loadtxt('label.csv', delimiter=',')

# Convert numpy array to torch tensor
x = torch.tensor(data.x, dtype=torch.float)
num_nodes = x.size(0)
edge_index = torch.randint(0, num_nodes, (2, 1000), dtype=torch.long)

# Create a PyTorch Geometric Data object
data = Data(x=x, edge_index=edge_index)

# Add train_mask and test_mask
num_nodes = x.size(0)
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[:int(0.8 * num_nodes)] = True
test_mask = ~train_mask

data.train_mask = train_mask
data.test_mask = test_mask

# Add labels (y) if not already present
num_classes = 3
data.y = torch.randint(0, num_classes, (num_nodes,), dtype=torch.long)

# Now you can access num_node_features
# gat_model = GAT(in_channels=data.num_node_features, out_channels=3)

/var/folders/yf/20z1hn994jd04q4kl0gpgh740000gn/T/ipykernel_88315/32871360.py:11: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [15]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.transforms import NormalizeFeatures
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import json

# Import GraphTransformer from core_rec.py
from core_rec import GraphTransformer 
import pandas as pd

# Normalize features
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

# Add train_mask and test_mask
num_nodes = data.num_nodes
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[:int(0.8 * num_nodes)] = True
test_mask = ~train_mask

data.train_mask = train_mask
data.test_mask = test_mask

# Add labels (y) if not already present
num_classes = dataset.num_classes

class GAT(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, 8, heads=8, dropout=0.6)
        self.conv2 = GATConv(8 * 8, 8, heads=8, dropout=0.6)
        self.conv3 = GATConv(8 * 8, out_channels, heads=1, concat=True, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv2(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv3(x, edge_index)
        return F.log_softmax(x, dim=1)

class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Define MetaExploitModel
class MetaExploitModel(torch.nn.Module):
    def __init__(self, input_dim):
        super(MetaExploitModel, self).__init__()
        num_layers = 1
        d_model = 128
        num_heads = 8
        d_feedforward = 512
        self.model = GraphTransformer(num_layers, d_model, num_heads, d_feedforward, input_dim, use_weights=True)
    def forward(self, data):
            adj_matrix = data.x.numpy()  # Assuming data.x is the adjacency matrix
            adj_matrix = torch.tensor(adj_matrix, dtype=torch.float32)
            output = self.model(adj_matrix)
            return F.log_softmax(output, dim=1)

def train(model, optimizer, is_meta_exploit=False):
    model.train()
    optimizer.zero_grad()
    if is_meta_exploit:
        out = model(data)
    else:
        out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    print(f"Training Loss: {loss.item()}")
    return loss.item()

def evaluate(model, metrics, is_meta_exploit=False):
    model.eval()
    with torch.no_grad():
        if is_meta_exploit:
            out = model(data)
            out = out.view(data.num_nodes, -1)  # Ensure the output shape matches the number of nodes and classes
        else:
            out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        y_true = data.y.cpu().numpy()
        y_pred = pred.cpu().numpy()

        precision = precision_score(y_true[data.test_mask], y_pred[data.test_mask], average='macro')
        recall = recall_score(y_true[data.test_mask], y_pred[data.test_mask], average='macro')
        f1 = f1_score(y_true[data.test_mask], y_pred[data.test_mask], average='macro')

        # Check if y_true and out have the same number of classes
        num_classes = out.shape[1]
        unique_labels = np.unique(y_true[data.test_mask])
        if len(unique_labels) == num_classes:
            roc_auc = roc_auc_score(y_true[data.test_mask], F.softmax(out[data.test_mask], dim=1).cpu().numpy(), multi_class='ovr')
        else:
            roc_auc = float('nan')  # Or handle this case as appropriate

        mcc = matthews_corrcoef(y_true[data.test_mask], y_pred[data.test_mask])
        accuracy = accuracy_score(y_true[data.test_mask], y_pred[data.test_mask])

        cm = confusion_matrix(y_true[data.test_mask], y_pred[data.test_mask])
        specificity = np.mean(np.diag(cm) / np.sum(cm, axis=1))
        sensitivity = np.mean(np.diag(cm) / np.sum(cm, axis=0))

        # metrics['precision'].append(precision)
        # metrics['recall'].append(recall)
        # metrics['f1_score'].append(f1)
        # metrics['specificity'].append(specificity)
        # metrics['sensitivity'].append(sensitivity)
        # metrics['roc_auc'].append(roc_auc)
        # metrics['mcc'].append(mcc)
        metrics['accuracy'].append(accuracy)


# Initialize models and optimizers with the correct number of output classes
num_classes = dataset.num_classes

gat_model = GAT(in_channels=data.num_node_features, out_channels=num_classes)
gat_optimizer = torch.optim.Adam(gat_model.parameters(), lr=0.01, weight_decay=5e-4)
gat_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(gat_optimizer, mode='min', factor=0.1, patience=5, verbose=True)

gcn_model = GCN(in_channels=data.num_node_features, out_channels=num_classes)
gcn_optimizer = torch.optim.Adam(gcn_model.parameters(), lr=0.01, weight_decay=5e-4)
gcn_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(gcn_optimizer, mode='min', factor=0.1, patience=5, verbose=True)

graph_sage_model = GraphSAGE(in_channels=data.num_node_features, hidden_channels=16, out_channels=num_classes)
graph_sage_optimizer = torch.optim.Adam(graph_sage_model.parameters(), lr=0.01, weight_decay=5e-4)
graph_sage_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(graph_sage_optimizer, mode='min', factor=0.1, patience=5, verbose=True)

meta_exploit_model = MetaExploitModel(input_dim=data.num_node_features)
meta_exploit_optimizer = torch.optim.Adam(meta_exploit_model.parameters(), lr=0.01, weight_decay=5e-4)
meta_exploit_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(meta_exploit_optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# Metrics storage
gat_metrics = {
    # 'precision': [],
    # 'recall': [],
    # 'f1_score': [],
    # 'specificity': [],
    # 'sensitivity': [],
    # 'roc_auc': [],
    # 'mcc': [],
    'accuracy': []
}

gcn_metrics = {
    # 'precision': [],
    # 'recall': [],
    # 'f1_score': [],
    # 'specificity': [],
    # 'sensitivity': [],
    # 'roc_auc': [],
    # 'mcc': [],
    'accuracy': []
}

graph_sage_metrics = {
    # 'precision': [],
    # 'recall': [],
    # 'f1_score': [],
    # 'specificity': [],
    # 'sensitivity': [],
    # 'roc_auc': [],
    # 'mcc': [],
    'accuracy': []
}

meta_exploit_metrics = {
    # 'precision': [],
    # 'recall': [],
    # 'f1_score': [],
    # 'specificity': [],
    # 'sensitivity': [],
    # 'roc_auc': [],
    # 'mcc': [],
    'accuracy': []
}

# Early stopping parameters
patience = 10
best_loss = float('inf')
patience_counter = 0
epoch=200
# Train and evaluate GAT model
for epoch in range(epoch):
    loss = train(gat_model, gat_optimizer)
    evaluate(gat_model, gat_metrics)
    print(f'GAT Epoch {epoch}, Loss: {loss}, Accuracy: {gat_metrics["accuracy"][-1]}')
    gat_scheduler.step(loss)

# Train and evaluate GCN model
for epoch in range(epoch):
    loss = train(gcn_model, gcn_optimizer)
    evaluate(gcn_model, gcn_metrics)
    print(f'GCN Epoch {epoch}, Loss: {loss}, Accuracy: {gcn_metrics["accuracy"][-1]}')
    gcn_scheduler.step(loss)

# Train and evaluate GraphSAGE model
for epoch in range(epoch):
    loss = train(graph_sage_model, graph_sage_optimizer)
    evaluate(graph_sage_model, graph_sage_metrics)
    print(f'GraphSAGE Epoch {epoch}, Loss: {loss}, Accuracy: {graph_sage_metrics["accuracy"][-1]}')
    graph_sage_scheduler.step(loss)

# Train and evaluate MetaExploitModel with early stopping and learning rate scheduler
for epoch in range(epoch):
    loss = train(meta_exploit_model, meta_exploit_optimizer, is_meta_exploit=True)
    evaluate(meta_exploit_model, meta_exploit_metrics, is_meta_exploit=True)
    print(f'MetaExploitModel Epoch {epoch}, Loss: {loss}, Accuracy: {meta_exploit_metrics["accuracy"][-1]}')

    # Update learning rate scheduler
    meta_exploit_scheduler.step(loss)


# Save metrics to JSON file
metrics_dict = {
    'GAT': gat_metrics,
    'GCN': gcn_metrics,
    'GraphSAGE': graph_sage_metrics,
    'MetaExploitModel': meta_exploit_metrics
}
with open('metrics.json', 'w') as f:
    json.dump(metrics_dict, f)

# Plotting the accuracy metrics
plt.figure(figsize=(12, 8))
plt.plot(gat_metrics['accuracy'], label='GAT Accuracy')
plt.plot(gcn_metrics['accuracy'], label='GCN Accuracy')
plt.plot(graph_sage_metrics['accuracy'], label='GraphSAGE Accuracy')
plt.plot(meta_exploit_metrics['accuracy'], label='CoreRec Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy over Epochs')   
plt.show()

/Users/visheshyadav/anaconda3/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning:

The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.

/Users/visheshyadav/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/var/folders/yf/20z1hn994jd04q4kl0gpgh740000gn/T/ipykernel_88315/4155612595.py:129: RuntimeWarning:

invalid value encountered in divide

/Users/visheshyadav/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/var/folders/yf/20z1hn994jd04q4kl0gpgh740000gn/T/ipykernel_88315/4155612595.py:129: RuntimeWarning:

invalid value encountered in divid

Training Loss: 1.944578766822815
GAT Epoch 0, Loss: 1.944578766822815, Accuracy: 0.3173431734317343
Training Loss: 1.8974978923797607
GAT Epoch 1, Loss: 1.8974978923797607, Accuracy: 0.3173431734317343
Training Loss: 1.854889154434204
GAT Epoch 2, Loss: 1.854889154434204, Accuracy: 0.3173431734317343
Training Loss: 1.8467940092086792
GAT Epoch 3, Loss: 1.8467940092086792, Accuracy: 0.3173431734317343


/Users/visheshyadav/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/var/folders/yf/20z1hn994jd04q4kl0gpgh740000gn/T/ipykernel_88315/4155612595.py:129: RuntimeWarning:

invalid value encountered in divide

/Users/visheshyadav/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/var/folders/yf/20z1hn994jd04q4kl0gpgh740000gn/T/ipykernel_88315/4155612595.py:129: RuntimeWarning:

invalid value encountered in divide

/Users/visheshyadav/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predi

Training Loss: 1.830358624458313
GAT Epoch 4, Loss: 1.830358624458313, Accuracy: 0.3173431734317343
Training Loss: 1.7960352897644043
GAT Epoch 5, Loss: 1.7960352897644043, Accuracy: 0.3173431734317343
Training Loss: 1.7617262601852417
GAT Epoch 6, Loss: 1.7617262601852417, Accuracy: 0.32287822878228783
Training Loss: 1.7494189739227295


/Users/visheshyadav/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/var/folders/yf/20z1hn994jd04q4kl0gpgh740000gn/T/ipykernel_88315/4155612595.py:129: RuntimeWarning:

invalid value encountered in divide

/Users/visheshyadav/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/var/folders/yf/20z1hn994jd04q4kl0gpgh740000gn/T/ipykernel_88315/4155612595.py:129: RuntimeWarning:

invalid value encountered in divide

/Users/visheshyadav/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predi

...

Training Loss: 1.7383627891540527
MetaExploitModel Epoch 136, Loss: 1.7383627891540527, Accuracy: 0.3173431734317343


/Users/visheshyadav/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/var/folders/yf/20z1hn994jd04q4kl0gpgh740000gn/T/ipykernel_88315/4155612595.py:129: RuntimeWarning:

invalid value encountered in divide



Training Loss: 1.7409933805465698
MetaExploitModel Epoch 137, Loss: 1.7409933805465698, Accuracy: 0.3173431734317343


/Users/visheshyadav/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/var/folders/yf/20z1hn994jd04q4kl0gpgh740000gn/T/ipykernel_88315/4155612595.py:129: RuntimeWarning:

invalid value encountered in divide



KeyboardInterrupt: 

In [ ]:

# Generate benchmark table in markdown
benchmark_table = """
# Benchmark Table

| Epoch Range | GAT Accuracy | GCN Accuracy | GraphSAGE Accuracy | MetaExploitModel Accuracy |
|-------------|--------------|--------------|--------------------|---------------------------|
"""

for start_epoch in range(0, epoch, 10):
    end_epoch = start_epoch + 10
    gat_acc = sum(gat_metrics['accuracy'][start_epoch:end_epoch]) / 10
    gcn_acc = sum(gcn_metrics['accuracy'][start_epoch:end_epoch]) / 10
    graph_sage_acc = sum(graph_sage_metrics['accuracy'][start_epoch:end_epoch]) / 10
    meta_exploit_acc = sum(meta_exploit_metrics['accuracy'][start_epoch:end_epoch]) / 10
    benchmark_table += f"| {start_epoch + 1}-{end_epoch} | {gat_acc:.3f} | {gcn_acc:.3f} | {graph_sage_acc:.3f} | {meta_exploit_acc:.3f} |\n"

print(benchmark_table)


# Benchmark Table

| Epoch Range | GAT Accuracy | GCN Accuracy | GraphSAGE Accuracy | MetaExploitModel Accuracy |
|-------------|--------------|--------------|--------------------|---------------------------|
| 1-10 | 0.370 | 0.317 | 0.211 | 0.354 |
| 11-20 | 0.676 | 0.318 | 0.337 | 0.382 |
| 21-30 | 0.843 | 0.337 | 0.479 | 0.616 |
| 31-40 | 0.882 | 0.436 | 0.631 | 0.692 |
| 41-50 | 0.892 | 0.532 | 0.761 | 0.736 |
| 51-60 | 0.891 | 0.627 | 0.841 | 0.753 |
| 61-70 | 0.893 | 0.709 | 0.861 | 0.771 |
| 71-80 | 0.893 | 0.757 | 0.872 | 0.771 |
| 81-90 | 0.893 | 0.773 | 0.880 | 0.766 |
| 91-100 | 0.893 | 0.785 | 0.884 | 0.767 |
| 101-110 | 0.893 | 0.804 | 0.885 | 0.768 |
| 111-120 | 0.893 | 0.824 | 0.888 | 0.765 |
| 121-130 | 0.893 | 0.845 | 0.886 | 0.764 |
| 131-140 | 0.893 | 0.858 | 0.886 | 0.762 |
| 141-150 | 0.893 | 0.865 | 0.886 | 0.767 |
| 151-160 | 0.893 | 0.865 | 0.886 | 0.769 |
| 161-170 | 0.893 | 0.866 | 0.886 | 0.772 |
| 171-180 | 0.893 | 0.867 | 0.886 | 0.771 |
| 181-190 | 0.893 

In [10]:
import plotly.graph_objects as go

# Data
epoch_range = [f'{i}-{i+9}' for i in range(1, 201, 10)]
gat_accuracy = [0.370, 0.676, 0.843, 0.882, 0.892, 0.891, 0.893, 0.893, 0.893, 0.893, 0.893, 0.893, 0.893, 0.893, 0.893, 0.893, 0.893, 0.893, 0.893, 0.893]
gcn_accuracy = [0.317, 0.318, 0.337, 0.436, 0.532, 0.627, 0.709, 0.757, 0.773, 0.785, 0.804, 0.824, 0.845, 0.858, 0.865, 0.865, 0.866, 0.867, 0.867, 0.780]
graphsage_accuracy = [0.211, 0.337, 0.479, 0.631, 0.761, 0.841, 0.861, 0.872, 0.880, 0.884, 0.885, 0.888, 0.886, 0.886, 0.886, 0.886, 0.886, 0.886, 0.886, 0.708]
metaexploitmodel_accuracy = [0.354, 0.382, 0.616, 0.692, 0.736, 0.753, 0.771, 0.771, 0.766, 0.767, 0.768, 0.765, 0.764, 0.762, 0.767, 0.769, 0.772, 0.771, 0.771, 0.540]

# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=epoch_range, y=gat_accuracy, mode='lines+markers', name='GAT Accuracy'))
fig.add_trace(go.Scatter(x=epoch_range, y=gcn_accuracy, mode='lines+markers', name='GCN Accuracy'))
fig.add_trace(go.Scatter(x=epoch_range, y=graphsage_accuracy, mode='lines+markers', name='GraphSAGE Accuracy'))
fig.add_trace(go.Scatter(x=epoch_range, y=metaexploitmodel_accuracy, mode='lines+markers', name='MetaExploitModel Accuracy'))

# Update layout
fig.update_layout(
    title='Accuracy Over Epochs',
    xaxis_title='Epoch Range',
    yaxis_title='Accuracy',
    legend_title='Models'
    # template='plotly_dark'
)

# Show plot
fig.show()